# Example Test Analysis
In this notebook, we feed well-known enzyme-substrate pairs from industrial applications to see if our model predicts them to be active

### Import packages

In [85]:
# General
import os
from os.path import join
CURRENT_DIR = os.getcwd()
import pickle

# Wrangling
import pandas as pd
import numpy as np

# Featurisation
import torch
from rdkit.Chem import MolFromSmiles, rdFingerprintGenerator

In [86]:
enzymes = {
    'enzyme' : ['laccase', 'cytochrome', 'lipase'],
    'sequence' : ['MFPGARILATLTLALHLLHGAHAAIGPAGNMYIVNEDVSPDGFARSAVVARSVPATDPTPATASIPGVLVQGNKGDNFQLNVVNQLSDTTMLKTTSIHWHGFFQAGSSWADGPAFVTQCPVASGDSFLYNFNVPDQAGTFWYHSHLSTQYCDGLRGPFVVYDPSDPHLSLYDIDNADTVITLEDWYHIVAPQNAAIPTPDSTLINGKGRYAGGPTSPLAIINVESNKRYRFRLVSMSCDPNFTFSIDGHSLLVIEADAVNIVPITVDSIQIFAGQRYSFVLTANQAVDNYWIRANPNLGSTGFVGGINSAILRYAGATEDDPTTTSSTSTPLLETNLVPLENPGAPGPPVPGGADININLAMAFDFTTFELTINGVPFLPPTAPVLLQILSGASTAASLLPSGSIYELEANKVVEISMPALAVGGPHPFHLHGHTFDVIRSAGSTTYNFDTPARRDVVNTGTGANDNVTIRFVTDNPGPWFLHCHIDWHLEIGLAVVFAEDVTSISAPPAAWDDLCPIYNALSDNDKGGIVPS',
                  'MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPFLGNILSYHKGFCMFDMECHKKYGKVWGFYDGQQPVLAITDPDMIKTVLVKECYSVFTNRRPFGPVGFMKSAISIAEDEEWKRLRSLLSPTFTSGKLKEMVPIIAQYGDVLVRNLRREAETGKPVTLKDVFGAYSMDVITSTSFGVNIDSLNNPQDPFVENTKKLLRFDFLDPFFLSITVFPFLIPILEVLNICVFPREVTNFLRKSVKRMKESRLEDTQKHRVDFLQLMIDSQNSKETESHKALSDLELVAQSIIFIFAGYETTSSVLSFIMYELATHPDVQQKLQEEIDAVLPNKAPPTYDTVLQMEYLDMVVNETLRLFPIAMRLERVCKKDVEINGMFIPKGVVVMIPSYALHRDPKYWTEPEKFLPERFSKKNKDNIDPYIYTPFGSGPRNCIGMRFALMNMKLALIRVLQNFSFKPCKETQIPLKLSLGGLLQPEKPVVLKVESRDGTVSGA',
                  'MKLLSLTGVAGVLATCVAATPLVKRLPSGSDPAFSQPKSVLDAGLTCQGASPSSVSKPILLVPGTGTTGPQSFDSNWIPLSTQLGYTPCWISPPPFMLNDTQVNTEYMVNAITALYAGSGNNKLPVLTWSQGGLVAQWGLTFFPSIRSKVDRLMAFAPDYKGTVLAGPLDALAVSAPSVWQQTTGSALTTALRNAGGLTQIVPTTNLYSATDEIVQPQVSNSPLDSSYLFNGKNVQAQAVCGPLFVIDHAGSLTSQFSYVVGRSALRSTTGQARSADYGITDCNPLPANDLTPEQKVAAAALLAPAAAAIVAGPKQNCEPDLMPYARPFAVGKRTCSGIVTP']
}      

In [87]:
substrates = {
    'substrate' : ['lignin', 'midazolam', 'trygliceride'],
    'smile' : ['CC(=O)NC1=C2C(=C(C=C1)S(=O)(=O)[O-])C=C(C(=C2O)N=NC3=CC=CC=C3)S(=O)(=O)[O-].[Na+].[Na+]',
                'CC1=NC=C2N1C3=C(C=C(C=C3)Cl)C(=NC2)C4=CC=CC=C4F',
                'C(C(COC=O)OC=O)OC=O']
}

In [88]:
enzyme_df = pd.DataFrame(enzymes)
substrate_df = pd.DataFrame(substrates)

In [89]:
# This code is inspired by: https://github.com/AlexanderKroll/ESP

# Set output path for fasta file of enzyme sequences
output_file_path = join(CURRENT_DIR, '..' ,'data', 'enzyme_data', 'example_sequences.fasta')

# Open the fasta file so we can write sequences to it
with open(join(CURRENT_DIR, '..' ,'Data', 'example_sequences.fasta'), 'w') as ofile:
    # Iterate over each row in the dataframe, taking sequences of each enzyme
    for index, row in enzyme_df.iterrows():
        seq = row['sequence']
        if not pd.isnull(seq):
            # Write the sequence in fasta format to the fasta file which includes the index of the given
            # enzyme in the enzyme_df dataframe, followed by its amino acid sequence
            ofile.write('>' + str(index) + '\n' + seq[:1018]  + '\n')

print(f'FASTA file created at {output_file_path}')

FASTA file created at /Users/pablocanocarciofa/Library/Mobile Documents/com~apple~CloudDocs/Masters/Project/Github/Enzyme-Specificity/Code/../data/enzyme_data/example_sequences.fasta


##### Command Line Code
python Code/extract.py esm1b_t33_650M_UR50S "/Users/pablocanocarciofa/Library/Mobile Documents/com~apple~CloudDocs/Masters/Project/Github/Enzyme-Specificity/Data/example_sequences.fasta" "/Users/pablocanocarciofa/Library/Mobile Documents/com~apple~CloudDocs/Masters/Project/Github/Enzyme-Specificity/Data/ESM_1b_examples" --repr_layers 33 --include mean

In [90]:
def load_embedding(index, model):
    """
    Load the embeddings created by an ESM model, in .pt files back into python

    Parameters:
    (1) index: the index in the enzyme_df dataframe, of the enzyme we want to retrieve (integer)
    (2) model: the ESM model we want to run - ESM-1b or ESM-2 (string)

    Returns:
    (1) embedding: vector representation of the given enzyme (list length 1280)
    """
    # Try to avoid throwing errors
    try:
        # Go to .pt file which is named after the index it takes in enzyme_df
        embedding_path = join(CURRENT_DIR, '..', 'Data', model, f'{index}.pt')
        # Load the file with torch
        embedding = torch.load(embedding_path)
        # Go into the file and grab the representation at the 33rd layer in the network
        # Convert to a numpy array then a list
        embedding = embedding['mean_representations'][33].numpy().tolist()
        return embedding
    # Don't throw error if file not found
    except FileNotFoundError:
        pass

# Create pandas series of embeddings, indexed by enzyme_df index
esm1b_series = pd.Series([load_embedding(int(idx), 'ESM-1b_examples') for idx in enzyme_df.index], index=enzyme_df.index)

# Assign the Series to a new column ESM1b in the DataFrame
enzyme_df['ESM1b'] = esm1b_series
print('We have', enzyme_df['ESM1b'].notna().sum(), 'enzymes with an ESM1b representation, out of', enzyme_df['sequence'].notna().sum(), 'enzymes with a sequence')

We have 3 enzymes with an ESM1b representation, out of 3 enzymes with a sequence


In [92]:
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=2048)

def string_to_float(list):
    """
    Convert all elements in a list to floats

    Parameters:
    (1) list: list for which we will convert all elements to a float (list)

    Returns:
    (1) floated_list: list where all elements have been converted to float (list)
    """
    floated_list = [float(element) for element in list]
    return floated_list

# Function to convert a SMILES string to a Morgan fingerprint bit string
def smiles_to_fingerprint(smiles):
    """
    Convert a SMILES string to an Extended Connectivity Fingerprint (ECFP)

    Parameters:
    (1) smiles: string SMILES representation of substrate (string)

    Returns:
    (1) ecfp: vector representation of input substrate SMILES representation (list length 2048)
    """
    if smiles:
        # Create Mol object from SMILES string
        mol = MolFromSmiles(smiles)
        if mol:
            # Get ECFP representation by calling GetFingerprint and convert this to a string, we then
            # convert this to a string and then to a list, then we convert all of the elements to floats
            # since each element will be a string at first
            ecfp = string_to_float(list(mfpgen.GetFingerprint(mol).ToBitString()))
            return ecfp
    return None

# Create ECFP representations for every SMILES representation of the substrates in our dataset
substrate_df['ECFP'] = substrate_df['SMILES'].apply(smiles_to_fingerprint)

In [93]:
# Concatenate enzyme and substrate features
examples_df = pd.concat([enzyme_df, substrate_df], axis = 1)

In [94]:
def combine_vectors(row, col1, col2):
        """
        Concatenates 2 vectors in 2 columns to one larger vector in one column in a dataframe

        (1) row - row of dataframe
        (2) col1 - column of dataframe containing first vector to be unpacked
        (3) col2 - column fo dataframe containing second vector to be unpacked
        """
        return np.concatenate([row[col1], row[col2]])

In [95]:
def featurise_train(df, col1, col2):
        """
        Creates a numpy array of 2 concatenated vectors from a dataframe where each value represents
        a row in the dataframe

        (1) row - row of dataframe
        (2) col1 - column of dataframe containing first vector to be unpacked
        (3) col2 - column fo dataframe containing second vector to be unpacked
        """
        df.loc[:, (col1 + '_' + col2 + '_Combined')] = df.apply(combine_vectors, args = (col1, col2), axis=1)
        featurised = np.stack(df[col1 + '_' + col2 + '_Combined'].values)
        return featurised

In [96]:
# Featurise enzyme-substrate pairs
X_examples = featurise_train(examples_df, 'ESM1b', 'ECFP')

In [97]:
# Load our best-performing model
with open('../Data/Objects/ESP.pkl', 'rb') as model:
    model_examples = pickle.load(model)

In [117]:
# Get predicted label and probability of prediction
label, prob = model_examples.predict(X_examples)
reactions = ['Laccase and lignin', 'Cytochrome P450 CYP3A4 and midazolam', 'Lipase and triglyceride']
for label, prob, reaction in zip(label,prob,reactions):
    print(f'{reaction} predicted label {label} with probability {prob:.4f}')

Laccase and lignin predicted label 1 with probability 0.9719
Cytochrome P450 CYP3A4 and midazolam predicted label 1 with probability 0.9816
Lipase and trygliceride predicted label 1 with probability 0.9324
